# Experiment 3 MobileNetV2

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np

2025-05-23 04:40:31.148240: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-23 04:40:31.165305: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747975231.184556   20448 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747975231.190493   20448 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-23 04:40:31.210345: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
DATA_DIR = "/home/sagemaker-user/gesture-gaze-extension/datasets/ColumbiaGazeProcessed"
IMG_SIZE = (96, 96)
BATCH_SIZE = 32
NUM_CLASSES = len(os.listdir(os.path.join(DATA_DIR, 'train')))

# Data Aug
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    brightness_range=[0.7, 1.3],
    zoom_range=0.1,
    horizontal_flip=True
).flow_from_directory(
    os.path.join(DATA_DIR, 'train'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    os.path.join(DATA_DIR, 'val'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    os.path.join(DATA_DIR, 'test'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 4125 images belonging to 9 classes.
Found 912 images belonging to 9 classes.
Found 886 images belonging to 9 classes.


In [4]:
base_model = MobileNetV2(input_shape=IMG_SIZE + (3,), include_top=False, weights='imagenet')
base_model.trainable = True


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    ]
)

# Evaluate
test_loss, test_acc = model.evaluate(test_gen)
print(f"Test Accuracy: {test_acc:.4f}")

W0000 00:00:1747975270.399661   20448 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


/opt/conda/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
129/129 ━━━━━━━━━━━━━━━━━━━━ 31s 207ms/step - accuracy: 0.1599 - loss: 2.2877 - val_accuracy: 0.2467 - val_loss: 1.9344
Epoch 2/30
129/129 ━━━━━━━━━━━━━━━━━━━━ 25s 197ms/step - accuracy: 0.2477 - loss: 1.9139 - val_accuracy: 0.2588 - val_loss: 1.8144
Epoch 3/30
129/129 ━━━━━━━━━━━━━━━━━━━━ 26s 202ms/step - accuracy: 0.2742 - loss: 1.8228 - val_accuracy: 0.2961 - val_loss: 1.7285
Epoch 4/30
129/129 ━━━━━━━━━━━━━━━━━━━━ 27s 207ms/step - accuracy: 0.2835 - loss: 1.7677 - val_accuracy: 0.2829 - val_loss: 1.7023
Epoch 5/30
129/129 ━━━━━━━━━━━━━━━━━━━━ 27s 205ms/step - accuracy: 0.3150 - loss: 1.7056 - val_accuracy: 0.3059 - val_loss: 1.6716
Epoch 6/30
129/129 ━━━━━━━━━━━━━━━━━━━━ 26s 202ms/step - accuracy: 0.2931 - loss: 1.6920 - val_accuracy: 0.3059 - val_loss: 1.6267
Epoch 7/30
129/129 ━━━━━━━━━━━━━━━━━━━━ 26s 203ms/step - accuracy: 0.3163 - loss: 1.6807 - val_accuracy: 0.3268 - val_loss: 1.6275
Epoch 8/30
129/129 ━━━━━━━━━━━━━━━━━━━━ 26s 203ms/step - accuracy: 0.3355 - loss: 1